In [ ]:
from google.colab import files
files.upload()


In [1]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/dataset.zip"

with ZipFile(file_name,'r') as zip:
  zip.extractall()

In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [9]:
Xtrain = []
i=1
for filename in os.listdir('/content/dataset/Train/'):
    i=i+1
    if(i>0 and i<2000):
      Xtrain.append(img_to_array(load_img('/content/dataset/Train/'+filename)))
    if(i>2000):
      break;
    
      

Xtrain = np.array(Xtrain, dtype=float)

# Set up train and test data
#split = int(0.95*len(X))
#Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain



In [3]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [10]:
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)



In [11]:
# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=10, steps_per_epoch=10)

Epoch 1/10
10/10 [==============================] - 4s 431ms/step - loss: 0.0106
Epoch 2/10
10/10 [==============================] - 4s 420ms/step - loss: 0.0127
Epoch 3/10
10/10 [==============================] - 4s 417ms/step - loss: 0.0095
Epoch 4/10
10/10 [==============================] - 4s 410ms/step - loss: 0.0105
Epoch 5/10
10/10 [==============================] - 4s 421ms/step - loss: 0.0097
Epoch 6/10
10/10 [==============================] - 4s 422ms/step - loss: 0.0094
Epoch 7/10
10/10 [==============================] - 4s 415ms/step - loss: 0.0098
Epoch 8/10
10/10 [==============================] - 4s 416ms/step - loss: 0.0091
Epoch 9/10
10/10 [==============================] - 4s 415ms/step - loss: 0.0097
Epoch 10/10
10/10 [==============================] - 4s 415ms/step - loss: 0.0099


In [ ]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

In [12]:
color_me = []
i=1
for filename in os.listdir('/content/dataset/Test/'):
    color_me.append(img_to_array(load_img('/content/dataset/Test/'+filename)))
    i=i+1
    if(i>10):
      break
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("output/img_"+str(i)+".png", lab2rgb(cur))

In [14]:
color_me = []
color_me.append(img_to_array(load_img('/content/dataset/Test/'+'3r9Lxm.jpg')))

color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("output/img_23"+".png", lab2rgb(cur))